In [ ]:
import os
import glob
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class Generate_Dataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.data_path = glob.glob\
            (os.path.join(path,'input/*.npy'))  #读取data文件夹下所有.npy格式文件

    def __getitem__(self, index):
        data_path = self.data_path[index]
        data = np.load(data_path)      #读取输入数据
        tensor_data = torch.from_numpy(data)
        
        label_path = data_path.replace('input', 'label')
        label = np.load(label_path)    #读取标签数据
        tensor_label = torch.from_numpy(label)

        return tensor_data, tensor_label

    def __len__(self):
        return len(self.data_path)
# if __name__ == '__main__':
#     dataset = Generate_Dataset('./dataset(05.09)/')
    
#     train_size = int(len(dataset) * 0.9)
#     validate_size = int(len(dataset) - train_size)
#     train_dataset, validate_dataset = torch.utils.data\
#                 .random_split(dataset, [train_size, validate_size])

#     #print("读入数据个数为：", len(top_dataset))
#     train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
#     validate_loader = DataLoader(validate_dataset, batch_size=1, shuffle=True)
#     t = 0
#     for train, label in train_loader:
#         t += 1
#         # print(train.shape)
#         # print(label.shape)
#     # print('共有',t,'个训练集')
    
#     n = 0
#     for validate, label in validate_loader:
#         n += 1
#         # print(validate.shape)
#         # print(label.shape)
#     # print('共有',n,'个训练集')

In [ ]:
import numpy as np
data_path = "./dataset(05.09)/"
def dataset_split(data_path, batch_size):
    dataset = Generate_Dataset(data_path)
    print(len(dataset))
    train_size = int(len(dataset) * 0.9)
    validate_size = int(len(dataset) - train_size)
    train_dataset, validate_dataset = torch.utils.data.random_split(dataset, \
        [train_size, validate_size])
    print(train_dataset.shape)
    
    # train_dataset = np.squeeze(train_dataset)

    # for i in range(len(train_dataset)):
    #     print(train_dataset[i].shape)
    
test = dataset_split(data_path, 50)

In [ ]:
import numpy as np
import glob
import os

label_array = []

dataindex = './dataset(05.09)/label/'
for t in range(len(glob.glob(os.path.join(dataindex, '*.npy')))):
    data = np.load(os.path.join(dataindex, str(t)+'.npy'))
    # print(data.shape)
    data = np.squeeze(data)
    # print(data.shape)
    if np.all(data == 0):
        label_array.append(0)
    else:
        label_array.append(t)
print(len(label_array))
# print(label_array)

label_array = np.array(label_array).reshape(108, 92)
np.set_printoptions(threshold=np.inf)
print(label_array)
n = 0
nozerolabel = []
# print(label_array.shape[0])
# print(label_array.shape[1])
for i in range(label_array.shape[0]):
    for j in range(label_array.shape[1]):
        if label_array[i][j] == 0:
             continue
        else:
            n += 1
            nozerolabel.append(label_array[i][j])
print(n)
print(nozerolabel)
print(len(nozerolabel))
np.save('./dataset(05.09)/nozerolabelindex.npy',nozerolabel)

__数据集整合代码__

In [ ]:
import numpy as np

#数据集划分整合模块
def dataset_split(data_path, labelindex, batch_size, negative_sample_ratio):
    
    #计算一个batch中negative_sample和positive_sample的数量
    negative_sample_size = int(batch_size * negative_sample_ratio)
    print('negative_sample_size:',negative_sample_size)
    positive_sample_size = batch_size - negative_sample_size
    print('positive_sample_size:',positive_sample_size)

    #划分存储数据集（单个batch）
    

    print('labelindex:',np.load(labelindex))
    


if __name__ == '__main__':
    data_path = './dataset(05.09)/'
    labelindex = './dataset(05.09)/nozerolabelindex.npy'
    test = dataset_split(data_path, labelindex, 50, 0.75)


__保存positive_sample__

In [ ]:
import numpy as np
import os
dataindex = np.array(np.load('./dataset(05.09)/nozerolabelindex.npy'))
print(dataindex)
dataaddr = './dataset(05.09)/input/'
inputsaveaddr = './dataset(positive_negative)/input/'

#保存positive_sample
for i in range(len(dataindex)):
    # print(i)
    index = dataindex[i]
    print(index)
    inputdata = np.load(os.path.join(dataaddr, str(index)+'.npy'))
    np.save(os.path.join(inputsaveaddr,'positive_input/'+str(index)+'.npy'),inputdata)

    labeladdr = dataaddr.replace('input/','label/')
    labeldata = np.load(os.path.join(labeladdr, str(index)+'.npy'))
    labelsaveaddr = inputsaveaddr.replace('input/','label/')
    np.save(os.path.join(labelsaveaddr,'positive_label/'+str(index)+'.npy'),labeldata)

    #保存negative_sample
    negativesample_index = np.


__保存negative_sample__

In [ ]:
import numpy as np
import os
#生成寻址标签
np.set_printoptions(threshold=np.inf)
dataindex = np.arange(0,9936)
# print(data)
positivedataindex = np.load('./dataset(05.09)/nozerolabelindex.npy')
print(positivedataindex)
for i in range(positivedataindex.shape[0]):
    temp_value = positivedataindex[i]
    dataindex[temp_value] = 0

#保存negative_sample
inputindex = './dataset(05.09)/input/'
inputsaveaddr = './dataset(positive_negative)/input/'
num = 0
for t in range(len(dataindex)):
    temp = dataindex[t]
    if temp == 0:
        continue
    else:
        inputdata = np.load(os.path.join(inputindex,str(temp)+'.npy'))
        num += 1
        np.save(os.path.join(inputsaveaddr,'negative_input/'+str(temp)+'.npy'), inputdata)
        labelindex = inputindex.replace('input/','label/')
        labeldata = np.load(os.path.join(labelindex,str(temp)+'.npy'))
        labelsaveaddr = inputsaveaddr.replace('input/', 'label/')
        np.save(os.path.join(labelsaveaddr, 'negative_label/'+str(temp)+'.npy'), labeldata)

print('一共：',num, '个negative_sample!')




__IOU指标实现__

In [3]:
import numpy as np
from sklearn import metrics

y_true = np.array([[0, 1, 2],[0, 2, 1]])
print(y_true)
y_pred = np.array([[2, 1, 0],[1, 0, 1]])
print(y_pred)


def cal_cm(y_true, y_pred):
    y_true = y_true.reshape(1, -1).squeeze()
    y_pred = y_pred.reshape(1, -1).squeeze()

    cm = metrics.confusion_matrix(y_true, y_pred)
    return cm

cm = cal_cm(y_true, y_pred)
print('cm',cm)

[[0 1 2]
 [0 2 1]]
[[2 1 0]
 [1 0 1]]
cm [[0 1 1]
 [0 2 0]
 [2 0 0]]


In [5]:
import numpy as np
import math
import torch
temp = torch.randint(0, 3, (3, 3))
print('temp',temp) 
confusion = [2, 4, 5, 1] 

def cb_loss(temp, beta, confusion):

    confusion_ = [torch.sum(temp == i) for i in range(4)]

    weight = []
    weight_dice = []
    for i, n in zip(confusion_, confusion):
        if i == 0:
            weight.append(0)
            weight_dice.append(1)
        else:
            weight_dice.append(1)
            weight.append(((1.0 - beta) / (1.0 - math.pow(beta, n))))
    return weight, weight_dice


weight, weight_dice = cb_loss(temp, 0.99, confusion)
print('weight',weight)
print('weight_dice',weight_dice)

temp tensor([[0, 0, 0],
        [0, 1, 1],
        [0, 2, 0]])
weight [0.5025125628140701, 0.2537814064007222, 0.2040401995838798, 0]
weight_dice [1, 1, 1, 1]
